# Assignment 3: Bitcoin Transaction and Generation

## Introduction
In this assignment, you will delve into the practical aspects of Bitcoin. Your task will involve understanding Bitcoin transactions, specifically focusing on transaction scripts. You will apply your Python skills to solve a real-world challenge: accepting and spending Bitcoin funds.


## Background
In Bitcoin, funds are locked in a transaction output (TXO) using a script (`scriptPubKey`) that requires a specific input to unlock (i.e., spend).

As seen in class, some of the common types of `scriptPubKey` are:
- **P2PK (Pay-to-Public-Key)**, which specifies a public key (i.e., a verification key of a digital signature scheme) and requires a matching signature on the transaction.
- **Pay-to-Public-Key-Hash (P2PKH)**, which specifies a hash digest and requires a public key that hashes to the digest alongside a matching signature on the transaction. A P2PKH script is of the form: "OP\_DUP, OP\_HASH160, hash, OP\_EQUALVERIFY, OP\_CHECKSIG"

The corresponding addresses are called "P2PK *address*" and "P2PKH *address*" respectively because the owner of a public key can tell people to send money to their public key simply by telling them what `scriptPubKey` (of either form) to use in a new TXO.

The `scriptPubKey` can also express other types of conditions for spending the funds.

You can read more about the Bitcoin scripting language in the [course reading material](https://docs.google.com/document/d/e/2PACX-1vQdnBw-O_IyPgchLNAQ6xOdKiPAIhnOUt0UFlusvtpNc8DuOLpA65Kg99p7tvyaIMyUKoMSO3TuWTfO/pub) and specifically at [Bitcoin Script Wiki](https://en.bitcoin.it/wiki/Script).

## Overview
The course staff have designated some coins for you and locked them with a P2PKH script on the Bitcoin blockchain (or more precisely, a Bitcoin testnet called Signet, which behaves just like the real Bitcoin blockchain, but does not have monetary value).

Your goal is to accept the funds from one of the transaction output and move them to another account. To do so, you need to create a new transaction whose input is an unspent TXO containing the lock script (`scriptPubKey`), and the new TXO is another address (or script) where you want to send the Bitcoins.

The secret key to unlock your TXO was sent to you via email. Your challenge is to find the relevant TXO, understand how its locking script works and how to satisfy it using the secret text you were given, and create a new transaction that spends that TXO and sends the funds to another address.

Follow the steps below to complete the assignment.


## **Step 1: Setup Your Environment**
Ensure you have the necessary libraries installed.

In [ ]:
# Install python-bitcoinlib if not already installed
!pip install python-bitcoinlib

---

## **Step 2: Import Required Libraries**
Import the required modules to handle Bitcoin transactions and scripts.

In [ ]:
from bitcoin import SelectParams
from bitcoin.core import CScript, CMutableTransaction, CMutableTxOut, CMutableTxIn, lx, b2lx, COutPoint, CTransaction
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

---

## **Step 3: Configure Bitcoin Network**
Switch to the Signet network, which is one of the Bitcoin testnet blockchains.

In [ ]:
# Select the Signet network
SelectParams('signet')

---

## **Step 4: Identify The Course Funding Transaction**
Use the secret provided via email to identify your unique UTXO from the transaction broadcasted by the course staff.

**Find the Transaction ID:** Use [mempool.space](https://mempool.space/signet) to locate the transaction. Switch to the Signet chain by clicking the bitcoin logo in the navbar and selecting `Signet'. Then, search for the course's funding address:

**Funding Address:** `mrA7b7UavMvPL9CmUPCSs3FtTXrbWBN8Ut`

Go to the most recent transaction from this address. This transaction has many outputs, each of which is a P2PKH script for a different student in the class. Next, you will find which of these transactions is meant for you.

In [ ]:
from bitcoin.core import Hash160

# Replace the placeholder with your actual WIF-encoded private key
# WIF stands for Wallet Import Format,
# which is a user-friendly way to represent Bitcoin private keys.
# Rather than displaying a raw binary number,
# WIF encodes the private key with additional metadata—such as a version byte
# and a checksum—making it easier to copy, paste, and import into wallets.
# Additionally, the encoding uses Base58Check, a method designed to avoid
# confusing characters and detect errors,
# ensuring that even a minor typo is quickly identified as invalid.
secret_phrase = 'None'

# Generate your secret key from the secret
secret_key = CBitcoinSecret(secret_phrase)

# Compute your public key from your secret key
public_key = secret_key.pub

# Hash your public key and print it in hex
Hash160(public_key).hex()


---

## **Step 5: Identify your unique UTXO**
Find the transaction output whose script matches your public key hash, and plug it in below

In [ ]:
# Plug in the transaction ID and the index of the output in that transaction
txid = 'txid' # TODO: replace with actual txid
vout = -1 # TODO: replace with actual output index

---

## **Step 6: Create the Transaction to Spend Your UTXO**
Define the input and output for your new transaction. Send 1000 sats to the course's address and payback yourself what is left. Don't forget to leave some funds as a transaction fee, or else your transaction may not get confirmed. 500 sats should do it.

In [ ]:
from bitcoin.core.script import CScript, OP_DUP, OP_HASH160, OP_EQUALVERIFY, OP_CHECKSIG, SignatureHash, SIGHASH_ALL
# Create the transaction input
# The transaction input must link to the UTXO that you have previously located
# This effectively points to the specific scriptPubKey you are unlocking
txin = CMutableTxIn(COutPoint(lx(txid), vout))

# Define the amount to send, adjusted for transaction fees. Amounts are measured in sats (1 BTC = 100,000,000 sat)
amount_to_send_to_output_1 = 0  # todo: fill me in with correct amountamount_to_send_to_output_2 = 0  # todo: fill me in with correct amount

# Define the course's funding address
course_address = P2PKHBitcoinAddress('-- fill me in --')

# Get the scriptPubKey of the destnation from the course's funding address
# Remember the address is just an encoding of a scriptPubKey!
scriptPubKey_1 = course_address.to_scriptPubKey()

# Compute a P2PKH script for your address. While you can do this with a built-in function
# like to_scriptPubKey(), to get full points you'll need to write a raw bitcoin script.
# In the bitcoinlib library, scripts are represented as a list of operations. For example,
# the following is a script that evaluates to true when passed an input that hashes to the
# fixed string 0xffffffffffffffffffffffffffffffffffffffff: 
#
# hash_check_script = [OP_HASH160, b'\xff' * 20, OP_EQUALVERIFY]
#
# Note that the operation to push bytes onto the stack is done implicitely. The bytes are
# just included as an element in the list.

p2pkh_script = [] # todo: fill me in
scriptPubKey_2 = CScript(p2pkh_script) 

# Create the transaction outputs
txout_1 = CMutableTxOut(amount_to_send_to_output_1, scriptPubKey_1)
txout_2 = CMutableTxOut(amount_to_send_to_output_2, scriptPubKey_2)

# Create the transaction, by putting the input output transactions
tx = CMutableTransaction([txin], [txout_1, txout_2])

---

## **Step 7: Create Your Unlocking Script (scriptSig)**
Use your secret to unlock the funds. Write code to create a new transaction that spends your UTXO.

In [ ]:
from bitcoin.core.script import CScript, OP_DUP, OP_HASH160, OP_EQUALVERIFY, OP_CHECKSIG, SignatureHash, SIGHASH_ALL

# The scriptPubKey you are unlocking
p2pkh_script = [] # todo: fill me in
scriptPubKey = CScript(p2pkh_script)

# Generate the message to sign: hash digest of the new transaction and the scriptPubKey of the spent UTXO
messageToSign = SignatureHash(scriptPubKey, tx, 0, SIGHASH_ALL)

# Sign this message to authorize the spending
signature = secret_key.sign(messageToSign) + bytes([SIGHASH_ALL])

# Add the signature and public key to the scriptSig and link it to the transaction
tx.vin[0].scriptSig = CScript([signature, public_key])

---

## **Step 8: Broadcast the Transaction**
Convert your transaction to hexadecimal format and broadcast it using [mempool.space](https://mempool.space/signet/tx/push).

In [ ]:
# Serialize and print the transaction in hex
raw_tx = tx.serialize().hex()

print("Raw transaction in hex:", raw_tx)

print("to decode the transaction go to https://live.blockcypher.com/btc/decodetx/")


---

## **Submission**
1. Wait until your transaction is confirmed in the blockchain (this may take several minutes), and put its transaction ID below as proof of completion.
2. If you used external resources or generative AI, add these below.
3. Submit the completed notebook, including your code.

Good luck!

In [ ]:
trx_id = "None" #insert your transaction id

---

## **Acknowledgment of external references and generative AI**

_Provide links to any external references you used; and prompts, session links or session copies for any substantial use of generative AI._
